In [1]:
import nltk
import math
import string
import os

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
#from nltk.stem.snowball import SnowballStemmer
#nltk.download()

from collections import Counter
from operator import itemgetter
from functools import partial

Parsing documents and queries

In [2]:
index_marker = ".I"
header_marker = ".T"
authors_marker = ".A"
meta_marker = ".B"
annotation_marker = ".W"

index_type = "index"
header_type = "header"
authors_type = "authors"
meta_type = "meta"
annotation_type = "annotation"
unknown_type = "unknown"

line_types = {index_marker:"index", header_marker:"header", authors_marker:"authors", meta_marker:"meta", annotation_marker:"annotation"}

def get_line_type(line):
    if line.startswith(index_marker):
        return line_types[index_marker]
    elif line.startswith(header_marker):
        return line_types[header_marker]
    elif line.startswith(authors_marker):
        return line_types[authors_marker]
    elif line.startswith(meta_marker):
        return line_types[meta_marker]
    elif line.startswith(annotation_marker):
        return line_types[annotation_marker]
    else:
        return unknown_type


text_file_name = "data/cran.all.1400"
documents = []
with open(text_file_name, 'r') as file:
    docId = 0
    header = ""
    annotation = ""
    
    is_header = False
    is_annotation = False
    
    for line in file:
        line_type = get_line_type(line)
        if line_type == index_type:
            is_header = False
            is_annotation = False
            
            if (docId):
                document = (docId, header, annotation)
                documents.append(document)
            
            docId = line.split()[1]
            header = ""
            annotation = ""
        elif line_type == header_type:
            is_header = True
            is_annotation = False
        elif line_type == annotation_type:
            is_header = False
            is_annotation = True
        elif line_type == authors_type or line_type == meta_type:
            is_header = False
            is_annotation = False
        elif line_type == unknown_type:
            if is_header:
                header += line
            elif is_annotation:
                annotation += line
                
    document = (docId, header, annotation)
    documents.append(document)
    
    
queries_file_name = "data/cran.qry"
queries = []
with open(queries_file_name, 'r') as file:
    queryId = 0
    text = ""
    
    is_text = False
    
    for line in file:
        line_type = get_line_type(line)
        if line_type == index_type:
            is_text = False
            
            if (queryId):
                query = (queryId, text)
                queries.append(query)
            
            queryId = line.split()[1]
            text = ""
        elif line_type == annotation_type:
            is_annotation = True
        elif line_type == unknown_type:
            text += line
    
    query = (queryId, text)
    queries.append(query)

Normalize and build index

In [3]:
def build_index(documents, index_by, normalizer, log_stats=False):
    docs = dict()
    index = dict()
    
    average_doc_lenght = 0
    
    for document in documents:
        text = document[index_by]
        normalized_text = normalizer(text)
        
        doc_lenght = len(normalized_text)
        average_doc_lenght += doc_lenght
        
        docId = document[0]
        docs[docId] = doc_lenght
            
        words_in_doc = Counter(normalized_text)
        for word, count in words_in_doc.items():
            if word not in index:
                index[word] = []
            index[word].append((docId, count))
            
    average_doc_lenght /= len(documents)
    
    if log_stats:
        avg_pos_list_len = 0
        max_pos_list_len = 0
        
        for word in index:
            positions = index[word]
            pos_list_len = len(positions)
            avg_pos_list_len += pos_list_len
            if pos_list_len > max_pos_list_len:
                max_pos_list_len = pos_list_len
        
        dict_size = len(index)
        avg_pos_list_len /= dict_size
                
        print("Dictionary size:", dict_size)
        print("Max positions list size:", max_pos_list_len)
        print("Avg positions list size:", avg_pos_list_len)
        print("Avg document size:", average_doc_lenght)
        
    return (index, docs, average_doc_lenght)

Search

In [4]:
def search(query, normalizer, rsv_function, index, docs_index, doc_lenght_avg, rsv_limit=None):
    search_terms = normalizer(query)
    
    query_index = dict()
    query_frequencies = Counter(search_terms)
    for word, count in query_frequencies.items():
        query_index[word] = count
    
    
    hits = dict()
    for search_term in search_terms:
        if search_term in index:
            hits[search_term] = index[search_term]
            
    found_docs = dict()
    for search_term in hits:
        doc_tf = hits[search_term]
        for docId, tf in doc_tf:
            if docId not in found_docs:
                found_docs[docId] = []
            hit = (search_term, tf)
            found_docs[docId].append(hit)
    
    result = []
    for docId in found_docs:
        Ld = docs_index[docId]
        rsv = rsv_function(found_docs[docId], query_index, index, docs_index, doc_lenght_avg, Ld)
        if rsv_limit is not None and rsv >= rsv_limit:
            result.append((docId, rsv))
        elif rsv_limit is None:
            result.append((docId, rsv))
        
        
    result.sort(key=itemgetter(1), reverse=True)
    return result[:10]


def process_queries_to_file(filepath, queries, normalizer, rsv_function, index, docs, avg_doc_lenght, rsv_limit=None):
    with open(filepath, 'w') as file:
        for idx, query in enumerate(queries):
            search_result = search(query[1], normalizer, rsv_function, index, docs, avg_doc_lenght, rsv_limit)

            for docId, _ in search_result:
                file.write("{} {}\n".format(idx + 1, docId))

Verification

In [5]:
search_results_file_path = "data/search_results"

def print_check_result():
    os.chdir('data')
    %run eval.py
    os.chdir('../')

<b>Задание 1-4</b>

Все функции, необходимые для выполнения задания, описаны выше. Для нормализации будем использовать следующую функцию:

In [6]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
punctuation = [c for c in string.punctuation]


def normalize_text(text):
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and word not in punctuation]
    return words

def calculate_rsv_for_doc(hits, query_index, index, docs_index, doc_lenght_avg, Ld, k1, b):
    rsv = 0
    N = len(docs_index)
    for search_term, tf in hits:
        Nt = len(index[search_term])
        term_rsv = math.log(1 + (N - Nt + 0.5) / (Nt + 0.5)) * tf * (k1 + 1) / (k1 * (1 - b + b*Ld/doc_lenght_avg) + tf)
        rsv += term_rsv
    return rsv

Т.е. наша базовая нормализация будет включать: токенизацию, лемматизацию, отсеивание стоп-слов, отсеивание знаков пунктуации.

Строим индекс только по заголовкам и по тексту с выводом статистики:

In [7]:
# Index by header
header_index, header_docs, header_avg_doc_lenght = build_index(documents, 1, normalize_text, True)

# Index by annotation
annotation_index, annotation_docs, annotation_avg_doc_lenght = build_index(documents, 2, normalize_text, True)

Dictionary size: 1770
Max positions list size: 358
Avg positions list size: 6.126553672316384
Avg document size: 7.817142857142857
Dictionary size: 8864
Max positions list size: 711
Avg positions list size: 10.036665162454874
Avg document size: 91.12142857142857


Главный вывод, который я могу сделать: аннотация содержит небольшую прибавку полезных слов. Не смотря на то, что средняя длина аннотации более чем в 10 раз больше средней длины заголовка, количество полезных слов увеличилось всего в 2 раза.

Результаты поиска по заголовкам:

In [10]:
rsv_function = partial(calculate_rsv_for_doc, k1=1.2, b=0.75)
process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght)
print_check_result()

mean precision: 0.24400000000000005
mean recall: 0.35545110212716374
mean F-measure: 0.2893649494054299
MAP@10: 0.2789772892696173


Результаты поиска по аннотациям:

In [11]:
rsv_function = partial(calculate_rsv_for_doc, k1=1.2, b=0.75)
process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, annotation_index, annotation_docs, annotation_avg_doc_lenght)
print_check_result()

mean precision: 0.2933333333333334
mean recall: 0.4208525878704234
mean F-measure: 0.3457085578890056
MAP@10: 0.3514682518686488


<b>Задание 5</b>

Результаты поиска по всем показателям выглядят лучше в случае использования полного текста аннотаций. Проще всего конечно оценивать значение F-меры, т.к. идеальное значение равно 1. Очевидно оно возрасло. Но это лишь следствие того, что возрасли precision и recall.

Precision возрос на 20%, recall вырос на 18%. Т.е. в относительном выражении точность возрасла сильнее.

<b>Задание 6</b>

Попробуем улучшить поиск. Для этого, воспользуемся методом grid search. Проверять будем все пары значений из диапозона [1.2, 2.1] для k1 и [0, 1.0] для b. Улучшить будем пытаться значение F-меры.

Выбор наилучших значений параметров будем производить вручную.

Рассмотрим сперва поиск только по заголовкам:

In [42]:
k1_values = [k1 / 10 for k1 in range(12, 21)]
b_values = [b / 10 for b in range(11)]

for k1_value in k1_values:
    for b_value in b_values:
        print("k1=", k1_value, ", b=", b_value)
        rsv_function = partial(calculate_rsv_for_doc, k1=k1_value, b=b_value)
        process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght)
        print_check_result()

k1= 1.2 , b= 0.0
mean precision: 0.22800000000000026
mean recall: 0.33569645164444295
mean F-measure: 0.2715603078630363
MAP@10: 0.2583579701016209
k1= 1.2 , b= 0.1
mean precision: 0.23733333333333348
mean recall: 0.3459814629294543
mean F-measure: 0.28153900567815543
MAP@10: 0.2681982132359117
k1= 1.2 , b= 0.2
mean precision: 0.24000000000000016
mean recall: 0.34928112622911756
mean F-measure: 0.2845075688454865
MAP@10: 0.27072203605722134
k1= 1.2 , b= 0.3
mean precision: 0.24044444444444463
mean recall: 0.3501491224304471
mean F-measure: 0.2851077828734066
MAP@10: 0.2726301741272641
k1= 1.2 , b= 0.4
mean precision: 0.24222222222222237
mean recall: 0.35317822295954765
mean F-measure: 0.28736160578328185
MAP@10: 0.27528690966098374
k1= 1.2 , b= 0.5
mean precision: 0.24266666666666684
mean recall: 0.3528679776493024
mean F-measure: 0.2875711655966476
MAP@10: 0.2772449371518155
k1= 1.2 , b= 0.6
mean precision: 0.24311111111111128
mean recall: 0.3532280373426954
mean F-measure: 0.28800276

mean precision: 0.23911111111111127
mean recall: 0.34804655832788306
mean F-measure: 0.28347343009142134
MAP@10: 0.27010789311609423
k1= 1.7 , b= 0.3
mean precision: 0.24222222222222242
mean recall: 0.3523058175871423
mean F-measure: 0.2870724081075725
MAP@10: 0.27415878964754625
k1= 1.7 , b= 0.4
mean precision: 0.2444444444444446
mean recall: 0.3550246728059975
mean F-measure: 0.2895355454045077
MAP@10: 0.2767842991517595
k1= 1.7 , b= 0.5
mean precision: 0.24311111111111128
mean recall: 0.3533230041043289
mean F-measure: 0.28803432237571464
MAP@10: 0.2785502113602643
k1= 1.7 , b= 0.6
mean precision: 0.24266666666666678
mean recall: 0.3529519839613789
mean F-measure: 0.28759905805831293
MAP@10: 0.2777983679068335
k1= 1.7 , b= 0.7
mean precision: 0.24622222222222234
mean recall: 0.35871052171991663
mean F-measure: 0.2920076741649103
MAP@10: 0.2807192463816802
k1= 1.7 , b= 0.8
mean precision: 0.24266666666666678
mean recall: 0.35326466634072795
mean F-measure: 0.28770280830644945
MAP@10:

Наилучшее значение F-меры достигается при k1=1.7, b=0.7:

k1= 1.7 , b= 0.7
mean precision: 0.24622222222222234
mean recall: 0.35871052171991663
mean F-measure: 0.2920076741649103
MAP@10: 0.2807192463816802

Попробуем запустить grid search в окрестностях этих значений:

In [43]:
k1_values = [k1 / 100 for k1 in range(165, 176)]
b_values = [b / 100 for b in range(65, 76)]

for k1_value in k1_values:
    for b_value in b_values:
        print("k1=", k1_value, ", b=", b_value)
        rsv_function = partial(calculate_rsv_for_doc, k1=k1_value, b=b_value)
        process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght)
        print_check_result()

k1= 1.65 , b= 0.65
mean precision: 0.2435555555555556
mean recall: 0.35481618149224314
mean F-measure: 0.2888420252928523
MAP@10: 0.2787714131743232
k1= 1.65 , b= 0.66
mean precision: 0.24400000000000005
mean recall: 0.35545110212716374
mean F-measure: 0.2893649494054299
MAP@10: 0.2789549102768679
k1= 1.65 , b= 0.67
mean precision: 0.24400000000000005
mean recall: 0.35600665768271933
mean F-measure: 0.2895488687077791
MAP@10: 0.27957145096721814
k1= 1.65 , b= 0.68
mean precision: 0.2453333333333334
mean recall: 0.3578427968521918
mean F-measure: 0.29109496138074414
MAP@10: 0.2801938915483889
k1= 1.65 , b= 0.69
mean precision: 0.244888888888889
mean recall: 0.3572078762172711
mean F-measure: 0.2905720308720932
MAP@10: 0.28015121077237476
k1= 1.65 , b= 0.7
mean precision: 0.24400000000000008
mean recall: 0.35583221484160976
mean F-measure: 0.28949115527007524
MAP@10: 0.27975279807396203
k1= 1.65 , b= 0.71
mean precision: 0.2457777777777779
mean recall: 0.3581549661643611
mean F-measure: 

mean precision: 0.24400000000000005
mean recall: 0.35545110212716374
mean F-measure: 0.2893649494054299
MAP@10: 0.27889444164497074
k1= 1.7 , b= 0.66
mean precision: 0.24444444444444452
mean recall: 0.35656221323827486
mean F-measure: 0.2900455461208038
MAP@10: 0.279061335069007
k1= 1.7 , b= 0.67
mean precision: 0.2453333333333334
mean recall: 0.35764526598799423
mean F-measure: 0.2910295833201315
MAP@10: 0.28007043475826543
k1= 1.7 , b= 0.68
mean precision: 0.2453333333333334
mean recall: 0.3578427968521918
mean F-measure: 0.29109496138074414
MAP@10: 0.2803614400492707
k1= 1.7 , b= 0.69
mean precision: 0.24444444444444455
mean recall: 0.3564671354765304
mean F-measure: 0.29001408462031036
MAP@10: 0.279873432994597
k1= 1.7 , b= 0.7
mean precision: 0.24622222222222234
mean recall: 0.35871052171991663
mean F-measure: 0.2920076741649103
MAP@10: 0.2807192463816802
k1= 1.7 , b= 0.71
mean precision: 0.24533333333333343
mean recall: 0.3579080525841142
mean F-measure: 0.29111655008139276
MAP@1

mean precision: 0.24488888888888896
mean recall: 0.3572412255839539
mean F-measure: 0.29058306401150813
MAP@10: 0.27971605498166346
k1= 1.75 , b= 0.67
mean precision: 0.2453333333333334
mean recall: 0.3578427968521918
mean F-measure: 0.29109496138074414
MAP@10: 0.28080794210688387
k1= 1.75 , b= 0.68
mean precision: 0.24444444444444455
mean recall: 0.3564671354765304
mean F-measure: 0.29001408462031036
MAP@10: 0.27990517902634304
k1= 1.75 , b= 0.69
mean precision: 0.2457777777777779
mean recall: 0.3582660772754722
mean F-measure: 0.29154783908252735
MAP@10: 0.2803568545113518
k1= 1.75 , b= 0.7
mean precision: 0.24533333333333343
mean recall: 0.3579080525841142
mean F-measure: 0.29111655008139276
MAP@10: 0.2798183204557543
k1= 1.75 , b= 0.71
mean precision: 0.24533333333333343
mean recall: 0.3579080525841142
mean F-measure: 0.29111655008139276
MAP@10: 0.28011295106519973
k1= 1.75 , b= 0.72
mean precision: 0.24533333333333343
mean recall: 0.35769752626832474
mean F-measure: 0.291046884408

Получили те же результаты:

k1= 1.7 , b= 0.7
mean precision: 0.24622222222222234
mean recall: 0.35871052171991663
mean F-measure: 0.2920076741649103
MAP@10: 0.2807192463816802

Т.о. параметры k1=1.7, b=0.7 показывают себя лучше стандартных, но совсем незначительно. По крайней мере с точки зрения значения F-меры.

Теперь рассмотрим поиск по аннотациям:

In [44]:
k1_values = [k1 / 10 for k1 in range(12, 21)]
b_values = [b / 10 for b in range(11)]

for k1_value in k1_values:
    for b_value in b_values:
        print("k1=", k1_value, ", b=", b_value)
        rsv_function = partial(calculate_rsv_for_doc, k1=k1_value, b=b_value)
        process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, annotation_index, annotation_docs, annotation_avg_doc_lenght)
        print_check_result()

k1= 1.2 , b= 0.0
mean precision: 0.2662222222222222
mean recall: 0.3827936907161917
mean F-measure: 0.3140391012408992
MAP@10: 0.3080224622770918
k1= 1.2 , b= 0.1
mean precision: 0.2702222222222222
mean recall: 0.3871805446030455
mean F-measure: 0.3182973739800325
MAP@10: 0.31706551048402914
k1= 1.2 , b= 0.2
mean precision: 0.2786666666666665
mean recall: 0.40054334846584944
mean F-measure: 0.3286703001595354
MAP@10: 0.32739416029786417
k1= 1.2 , b= 0.3
mean precision: 0.2813333333333332
mean recall: 0.4036577692790532
mean F-measure: 0.3315733162783422
MAP@10: 0.33390169088211424
k1= 1.2 , b= 0.4
mean precision: 0.28755555555555545
mean recall: 0.4108082952111884
mean F-measure: 0.33830561941769194
MAP@10: 0.3409201464124747
k1= 1.2 , b= 0.5
mean precision: 0.2906666666666666
mean recall: 0.4177337586366517
mean F-measure: 0.3428040832275447
MAP@10: 0.3472757621567145
k1= 1.2 , b= 0.6
mean precision: 0.29244444444444445
mean recall: 0.41886295224623743
mean F-measure: 0.34441970922250

mean precision: 0.2737777777777777
mean recall: 0.3928406154815073
mean F-measure: 0.32267645722025234
MAP@10: 0.3198651010610008
k1= 1.7 , b= 0.2
mean precision: 0.27911111111111103
mean recall: 0.4011705818114737
mean F-measure: 0.3291905926600717
MAP@10: 0.3318384829652027
k1= 1.7 , b= 0.3
mean precision: 0.2844444444444444
mean recall: 0.407793526767752
mean F-measure: 0.33512927054946906
MAP@10: 0.33798066403516147
k1= 1.7 , b= 0.4
mean precision: 0.2888888888888888
mean recall: 0.41052762781557833
mean F-measure: 0.3391308824579728
MAP@10: 0.3471586461745192
k1= 1.7 , b= 0.5
mean precision: 0.29199999999999987
mean recall: 0.41642109163771
mean F-measure: 0.34328441147089833
MAP@10: 0.35223706433190577
k1= 1.7 , b= 0.6
mean precision: 0.2937777777777778
mean recall: 0.4224178135194894
mean F-measure: 0.3465449049322481
MAP@10: 0.3547917660479833
k1= 1.7 , b= 0.7
mean precision: 0.29377777777777775
mean recall: 0.4227122079288264
mean F-measure: 0.34664393239885555
MAP@10: 0.35425

Наилучшее значение F-меры достигается при k1=2.0, b=0.7:

k1= 2.0 , b= 0.7
mean precision: 0.29511111111111105
mean recall: 0.42523987540821795
mean F-measure: 0.3484218512057311
MAP@10: 0.35932742714369736

Попробуем запустить grid search в окрестностях этих значений:

In [45]:
k1_values = [k1 / 100 for k1 in range(195, 206)]
b_values = [b / 100 for b in range(65,76)]

for k1_value in k1_values:
    for b_value in b_values:
        print("k1=", k1_value, ", b=", b_value)
        rsv_function = partial(calculate_rsv_for_doc, k1=k1_value, b=b_value)
        process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, annotation_index, annotation_docs, annotation_avg_doc_lenght)
        print_check_result()

k1= 1.95 , b= 0.65
mean precision: 0.2955555555555556
mean recall: 0.42569899757260343
mean F-measure: 0.3488857108251973
MAP@10: 0.35863805814506894
k1= 1.95 , b= 0.66
mean precision: 0.29644444444444445
mean recall: 0.4256119280469373
mean F-measure: 0.34947490629704014
MAP@10: 0.35856548318916076
k1= 1.95 , b= 0.67
mean precision: 0.29600000000000004
mean recall: 0.4258045206395299
mean F-measure: 0.3492306698152268
MAP@10: 0.3584633723859917
k1= 1.95 , b= 0.68
mean precision: 0.2955555555555555
mean recall: 0.4252081293764719
mean F-measure: 0.3487207458751294
MAP@10: 0.3581143172083652
k1= 1.95 , b= 0.69
mean precision: 0.29511111111111105
mean recall: 0.4250493992177418
mean F-measure: 0.3483578971109615
MAP@10: 0.3583640526581006
k1= 1.95 , b= 0.7
mean precision: 0.29466666666666663
mean recall: 0.4246049547732973
mean F-measure: 0.347898965964255
MAP@10: 0.35882269043419873
k1= 1.95 , b= 0.71
mean precision: 0.29511111111111105
mean recall: 0.4258238099921526
mean F-measure: 0.

mean precision: 0.29644444444444445
mean recall: 0.4263688945372371
mean F-measure: 0.34972982166475053
MAP@10: 0.3593630392486215
k1= 2.0 , b= 0.67
mean precision: 0.29777777777777775
mean recall: 0.42794534278035196
mean F-measure: 0.3511879656954524
MAP@10: 0.35947707021080066
k1= 2.0 , b= 0.68
mean precision: 0.29555555555555557
mean recall: 0.4255573357256782
mean F-measure: 0.3488381259075075
MAP@10: 0.3592468526916943
k1= 2.0 , b= 0.69
mean precision: 0.29555555555555557
mean recall: 0.4255573357256782
mean F-measure: 0.3488381259075075
MAP@10: 0.3595274523389608
k1= 2.0 , b= 0.7
mean precision: 0.29511111111111105
mean recall: 0.42523987540821795
mean F-measure: 0.3484218512057311
MAP@10: 0.35932742714369736
k1= 2.0 , b= 0.71
mean precision: 0.29511111111111105
mean recall: 0.4258238099921526
mean F-measure: 0.34861770189198815
MAP@10: 0.3595297787016044
k1= 2.0 , b= 0.72
mean precision: 0.29511111111111105
mean recall: 0.4278959600643027
mean F-measure: 0.3493101443924746
MAP@

mean precision: 0.29600000000000004
mean recall: 0.4259277060960486
mean F-measure: 0.34927209453201635
MAP@10: 0.3598410913748219
k1= 2.05 , b= 0.68
mean precision: 0.29511111111111105
mean recall: 0.4250635085651844
mean F-measure: 0.3483626356114636
MAP@10: 0.3592929432266738
k1= 2.05 , b= 0.69
mean precision: 0.29555555555555557
mean recall: 0.4255573357256782
mean F-measure: 0.3488381259075075
MAP@10: 0.359597117241959
k1= 2.05 , b= 0.7
mean precision: 0.296
mean recall: 0.4259613761297186
mean F-measure: 0.34928341460677376
MAP@10: 0.3598635487528349
k1= 2.05 , b= 0.71
mean precision: 0.29555555555555557
mean recall: 0.42637407454241705
mean F-measure: 0.34911221598916287
MAP@10: 0.3599131134346747
k1= 2.05 , b= 0.72
mean precision: 0.296
mean recall: 0.42865786482620744
mean F-measure: 0.35018657534059144
MAP@10: 0.36046053092018726
k1= 2.05 , b= 0.73
mean precision: 0.29555555555555557
mean recall: 0.4281993110343203
mean F-measure: 0.3497225126997854
MAP@10: 0.3602294315668656

Значение F-меры удалось ещё немного улучшить. Наилучшее значение достигается при k1=2.05, b=0.66. 

k1= 2.05 , b= 0.66
mean precision: 0.2964444444444445
mean recall: 0.4266651908335333
mean F-measure: 0.349829456806525
MAP@10: 0.36055833613280736

Учитывая, что k1 равно краевому значению - возможно имеет смысл попробовать и большие значения k1. Но не думаю, что рост будет существенным (если будет).

В целом, рост значения F-меры по отношению к исходному составляет 0.9%. Мне кажется, что существенного прироста изменением параметров BM25 не добиться, и более предпочтительным является изменение способа обработки термов текста и запроса.

В целом, мне кажется, что оптимальные значения этих параметров каким-то образом зависят от процента содержания в тексте запроса и индекируемом тексте отсеиваемых слов. Т.е. стоп слов, пунктуации и т.п.

<b>Задание 7</b>

Попробуем формулу для вычисления IDF состовляющей в BM25 со слайда №34:

In [31]:
def calculate_rsv_for_doc_34(hits, query_index, index, docs_index, doc_lenght_avg, Ld, k1, b):
    rsv = 0
    N = len(docs_index)
    for search_term, tf in hits:
        Nt = len(index[search_term])
        term_rsv = math.log(N / Nt) * tf * (k1 + 1) / (k1 * (1 - b + b*Ld/doc_lenght_avg) + tf)
        rsv += term_rsv
    return rsv

Результаты для индекса только по заголовкам:

In [32]:
rsv_function = partial(calculate_rsv_for_doc_34, k1=1.2, b=0.75)
process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght)
print_check_result()

mean precision: 0.2431111111111112
mean recall: 0.35314951482557644
mean F-measure: 0.28797665719658627
MAP@10: 0.2779021611936956


Результаты для индекса по аннотациям:

In [33]:
rsv_function = partial(calculate_rsv_for_doc_34, k1=1.2, b=0.75)
process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, annotation_index, annotation_docs, annotation_avg_doc_lenght)
print_check_result()

mean precision: 0.2928888888888889
mean recall: 0.420482217500053
mean F-measure: 0.3452749021600567
MAP@10: 0.3512975875535399


В обоих случаях наблюдаются падения всех основных метрик, но крайне не значительные.

<b>Задание 8</b>

Попробуем нормировать rsv(q,d) на сумму IDF термов запроса. В качестве формулы для IDF будем использовать формулу IDF составляющей из формулы RSV со слайда 34 лекции 2.

In [19]:
def calculate_rsv_for_doc_norm_by_idf(hits, query_index, index, docs_index, doc_lenght_avg, Ld, k1, b):
    rsv = 0
    N = len(docs_index)
    for search_term, tf in hits:
        Nt = len(index[search_term])
        term_rsv = math.log(1 + (N - Nt + 0.5) / (Nt + 0.5)) * tf * (k1 + 1) / (k1 * (1 - b + b*Ld/doc_lenght_avg) + tf)
        rsv += term_rsv
        
    idf_sum = 0
    for query_term in query_index:
        Nqt = 0
        if query_term in index:
            Nqt = len(index[query_term])
        idf_sum += math.log(1 + (N - Nqt + 0.5) / (Nqt + 0.5))
        
    return rsv / idf_sum

In [18]:
rsv_function = partial(calculate_rsv_for_doc_norm_by_idf, k1=1.2, b=0.75)
process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght)
print_check_result()

mean precision: 0.24400000000000005
mean recall: 0.35545110212716374
mean F-measure: 0.2893649494054299
MAP@10: 0.2789772892696173


Результат не изменился. Действительно, ведь для каждого документа сумма IDF термов запроса одинаковая (т.к. зависит только от запроса), поэтому при нормировании по этому значению мы изменим RSV всех документов одинаково. 

Я вам писал об этом письмо.

<b>Задание 9</b>

Реализуем формулу BM25, которая учитывает частоту терма в запросе.

In [34]:
def calculate_rsv_for_doc_ftq(hits, query_index, index, docs_index, doc_lenght_avg, Ld, k1, k2, b):
    rsv = 0
    N = len(docs_index)
    for search_term, tf in hits:
        Nt = len(index[search_term])
        ftq = query_index[search_term]
        term_rsv = math.log(1 + (N - Nt + 0.5) / (Nt + 0.5)) * tf * (k1 + 1) / (k1 * (1 - b + b*Ld/doc_lenght_avg) + tf) * (k2 + 1) * ftq / (k2 + ftq)
        rsv += term_rsv
    return rsv

Попробуем подобрать оптимальное значение k2. Интерироваться будем по широкой сетке, в качестве метрики качества будем опираться на значение F-меры.

Для поиска по заголовкам:

In [37]:
k2_values = [0, 5, 10, 100, 300, 500, 800, 1000]
for k2_value in k2_values:
    print("k2=", k2_value)
    rsv_function = partial(calculate_rsv_for_doc_ftq, k1=1.2, k2=k2_value, b=0.75)
    process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght)
    print_check_result()

k2= 0
mean precision: 0.24400000000000005
mean recall: 0.35545110212716374
mean F-measure: 0.2893649494054299
MAP@10: 0.2789772892696173
k2= 5
mean precision: 0.24488888888888902
mean recall: 0.356738851414913
mean F-measure: 0.29041673145716956
MAP@10: 0.2806971683323535
k2= 10
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.28085384087791493
k2= 100
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.28045174967106185
k2= 300
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.28045174967106185
k2= 500
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.28045174967106185
k2= 800
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.28045174967106185
k2= 1000
mean precision: 

Наилучшие значения достигаются при 0<k2<10. Т.е. при виличиная 1-2 порядков. Проведем оптимизацию по более узкой сетке.

In [38]:
k2_values = range(1, 21)
for k2_value in k2_values:
    print("k2=", k2_value)
    rsv_function = partial(calculate_rsv_for_doc_ftq, k1=1.2, k2=k2_value, b=0.75)
    process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght)
    print_check_result()

k2= 1
mean precision: 0.2444444444444446
mean recall: 0.35570181437787596
mean F-measure: 0.28976047463531085
MAP@10: 0.2813591822737325
k2= 2
mean precision: 0.24488888888888904
mean recall: 0.35614625882232037
mean F-measure: 0.2902201707739034
MAP@10: 0.28164348562470254
k2= 3
mean precision: 0.2444444444444446
mean recall: 0.35525736993343154
mean F-measure: 0.2896128987645627
MAP@10: 0.2813217085187984
k2= 4
mean precision: 0.24488888888888902
mean recall: 0.356738851414913
mean F-measure: 0.29041673145716956
MAP@10: 0.2818971683323535
k2= 5
mean precision: 0.24488888888888902
mean recall: 0.356738851414913
mean F-measure: 0.29041673145716956
MAP@10: 0.2806971683323535
k2= 6
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.2805734175974916
k2= 7
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.2805998726239467
k2= 8
mean precision: 0.24533333333333346
m

Выходит все же скорее все величины второго порядка. Проведем еще раз оптимизацию по величинам второго порядка.

In [39]:
k2_values = range(10, 100)
for k2_value in  k2_values:
    print("k2=", k2_value)
    rsv_function = partial(calculate_rsv_for_doc_ftq, k1=1.2, k2=k2_value, b=0.75)
    process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght)
    print_check_result()

k2= 10
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.28085384087791493
k2= 11
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.28070040172447575
k2= 12
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.2806663881190336
k2= 13
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.2806311147504269
k2= 14
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.2814247655440777
k2= 15
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.2814247655440777
k2= 16
mean precision: 0.24533333333333346
mean recall: 0.3574795921556537
mean F-measure: 0.2909747161477855
MAP@10: 0.2814247655440777
k2= 17
mean precision: 0.24533333333333

mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.2808221200414322
k2= 71
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.2808221200414322
k2= 72
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.2808221200414322
k2= 73
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.2808221200414322
k2= 74
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.2808221200414322
k2= 75
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.2808221200414322
k2= 76
mean precision: 0.24488888888888902
mean recall: 0.35673885141491296
mean F-measure: 0.29041673145716956
MAP@10: 0.2808221200414322
k2= 77
mean precision: 0.244888888

Итого оптимальное значение k2 лежит в пределах [7, 26], но если учесть также значения MAP@10, то [14, 24]. Однако относительный рост значения F-меры при этом составил всего 0.5%, а MAP@10 0.9%.

Теперь рассмотрим поиск по аннотации. Будем проделывать те же шаги.

In [46]:
k2_values = [0, 5, 10, 100, 300, 500, 800, 1000]
for k2_value in k2_values:
    print("k2=", k2_value)
    rsv_function = partial(calculate_rsv_for_doc_ftq, k1=1.2, k2=k2_value, b=0.75)
    process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, annotation_index, annotation_docs, annotation_avg_doc_lenght)
    print_check_result()

k2= 0
mean precision: 0.2933333333333334
mean recall: 0.4208525878704234
mean F-measure: 0.3457085578890056
MAP@10: 0.3514682518686488
k2= 5
mean precision: 0.2964444444444444
mean recall: 0.427083788768291
mean F-measure: 0.34997007906785843
MAP@10: 0.3531541572184433
k2= 10
mean precision: 0.29511111111111105
mean recall: 0.42382452950903177
mean F-measure: 0.3479458264488493
MAP@10: 0.3528192218862857
k2= 100
mean precision: 0.295111111111111
mean recall: 0.42340097775214663
mean F-measure: 0.3478030082629451
MAP@10: 0.3530428991349629
k2= 300
mean precision: 0.295111111111111
mean recall: 0.42340097775214663
mean F-measure: 0.3478030082629451
MAP@10: 0.35302173511379886
k2= 500
mean precision: 0.295111111111111
mean recall: 0.42340097775214663
mean F-measure: 0.3478030082629451
MAP@10: 0.35302173511379886
k2= 800
mean precision: 0.295111111111111
mean recall: 0.42340097775214663
mean F-measure: 0.3478030082629451
MAP@10: 0.35302173511379886
k2= 1000
mean precision: 0.29511111111111

Кажется, что оптимальное значение лежит в первой 20-ке.

In [47]:
k2_values = range(0, 20)
for k2_value in k2_values:
    print("k2=", k2_value)
    rsv_function = partial(calculate_rsv_for_doc_ftq, k1=1.2, k2=k2_value, b=0.75)
    process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, annotation_index, annotation_docs, annotation_avg_doc_lenght)
    print_check_result()

k2= 0
mean precision: 0.2933333333333334
mean recall: 0.4208525878704234
mean F-measure: 0.3457085578890056
MAP@10: 0.3514682518686488
k2= 1
mean precision: 0.29511111111111105
mean recall: 0.4259479862991552
mean F-measure: 0.34865930951776736
MAP@10: 0.35304132653061254
k2= 2
mean precision: 0.29466666666666663
mean recall: 0.4252072455584145
mean F-measure: 0.3481009648035066
MAP@10: 0.3523865940203245
k2= 3
mean precision: 0.2955555555555555
mean recall: 0.4259479862991552
mean F-measure: 0.34896930209046073
MAP@10: 0.3530480284706478
k2= 4
mean precision: 0.2964444444444444
mean recall: 0.427083788768291
mean F-measure: 0.34997007906785843
MAP@10: 0.35332721718316995
k2= 5
mean precision: 0.2964444444444444
mean recall: 0.427083788768291
mean F-measure: 0.34997007906785843
MAP@10: 0.3531541572184433
k2= 6
mean precision: 0.296
mean recall: 0.4261948998794021
mean F-measure: 0.34936189769650594
MAP@10: 0.35312923070462787
k2= 7
mean precision: 0.2955555555555555
mean recall: 0.4247

Действительно, наилучшие показатели достигаются при k2=4. При этом рост F-меры составил 1.2%, а MAP@10 0.5%.

Т.о. кажется, что за счет этого дополнительного компонента можно выиграть улучшение показателей в пределах 2%. При этом эффективность больше на большем объеме текста.

<b>Задание 10</b>

Чтобы найти ответ на поставленный вопрос - проитерируемся по предельным значениям RSV.

In [51]:
limits = [0.5, 1, 5, 10, 15, 20, 25]

rsv_function = partial(calculate_rsv_for_doc_34, k1=1.2, b=0.75)
for limit in limits:
    print("RSV limit=", limit)
    process_queries_to_file(search_results_file_path, queries, normalize_text, rsv_function, header_index, header_docs, header_avg_doc_lenght, limit)
    print_check_result()

RSV limit= 0.5
mean precision: 0.2431111111111112
mean recall: 0.35314951482557644
mean F-measure: 0.28797665719658627
MAP@10: 0.2779021611936956
RSV limit= 1
mean precision: 0.2431111111111112
mean recall: 0.35314951482557644
mean F-measure: 0.28797665719658627
MAP@10: 0.2779021611936956
RSV limit= 5
mean precision: 0.24574426807760147
mean recall: 0.3520384037144653
mean F-measure: 0.28944104250017694
MAP@10: 0.2789890218638896
RSV limit= 10
mean precision: 0.48599891963766295
mean recall: 0.2748014056816865
mean F-measure: 0.35108603882405337
MAP@10: 0.4678170217088194
RSV limit= 15
mean precision: 0.7805351006381935
mean recall: 0.23555733804845055
mean F-measure: 0.3618977241821868
MAP@10: 0.7672860415643923
RSV limit= 20
mean precision: 0.9054054054054054
mean recall: 0.22775301275301268
mean F-measure: 0.36395406950966225
MAP@10: 0.8969594594594594
RSV limit= 25
mean precision: 0.9642857142857143
mean recall: 0.17468511647083074
mean F-measure: 0.2957869644462627
MAP@10: 0.96428

Таким образом ответ на поставленный вопрос положительный - такой предел установить можно.